In [1]:
!pip install ta
!pip install matplotlib
!pip install --upgrade mplfinance

import sqlalchemy
import pandas as pd
import numpy as np
import yfinance as yf
import ta
from scipy.stats import linregress
import mplfinance as mpf
from datetime import datetime, date
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")


Requirement already up-to-date: mplfinance in c:\users\user\anaconda3\lib\site-packages (0.12.9b7)


In [2]:
def test_vcp(dfs):
    for df in dfs:
        df['Future_Close_1'] = df['Close'].shift(periods = -5)
        df['Future_Close_3'] = df['Close'].shift(periods = -10)
        df['Future_Close_5'] = df['Close'].shift(periods = -20)
        df['Future_Close_7'] = df['Close'].shift(periods = -50)
        df['Result_1'] = ( df['Future_Close_1'] - df['Close'] ) / df['Close']
        df['Result_3'] = ( df['Future_Close_3'] - df['Close'] ) / df['Close']
        df['Result_5'] = ( df['Future_Close_5'] - df['Close'] ) / df['Close']
        df['Result_7'] = ( df['Future_Close_7'] - df['Close'] ) / df['Close']
        vcp_date = df.loc[(df['Fufillment']== True)]
        print(vcp_date['Symbol'].tail(1).iloc[-1])
        print(vcp_date[['Result_1','Result_3','Result_5','Result_7']].describe())
        mpf.plot(frame.loc['2022':'2023',:],type='candle', style ='charles',volume = True, mav=(20,50,100))

In [3]:
class VCP_Screener:
    engine = sqlalchemy.create_engine('sqlite:///HK')
    
    #retreving symbol list from db
    def gettables(self):
        names = pd.read_sql('Select name from sqlite_master where type="table"',self.engine)
        names = names.name.to_list() 
        return names
    
    #get the prices base on the symbol
    def getprices(self):
        prices = []
        names = self.gettables()
        for name in names:
            prices.append(pd.read_sql(f'SELECT * From "{name}"',self.engine))
        return prices
   
    #update db process
    def maxdate(self):
        req = self.getprices()[0]
        req.Date =  pd.to_datetime(req.Date)
        return req.Date.max()
        
    def updateDB(self):
        maxdate = self.maxdate()
        engine = sqlalchemy.create_engine('sqlite:///HK')
        for symbol in self.gettables():
            data = yf.download(symbol, start = maxdate)
            data = data[data.index> maxdate]
            data = data.reset_index()
            data['Symbol'] = symbol
            data.to_sql(symbol,engine,if_exists='append',index=False)
        print("Database update sucessfully")

            
    #Calculate the slope for Strategy creiteria use 
    def slope_reg(self,arr):
        y = np.array(arr)
        x = np.arange(len(y))
        slope, intercept, r_value, p_value, std_err = linregress(x,y)
        return slope

    #calculate the rs score
    def rs_line(self,df):
        df['ROC63'] = df['Adj Close'].pct_change(periods=63)
        df['ROC126'] = df['Adj Close'].pct_change(periods=126)
        df['ROC189'] = df['Adj Close'].pct_change(periods=189)
        df['ROC252'] = df['Adj Close'].pct_change(periods=252)
        df['RS_line'] = round(2*df['ROC63']+df['ROC126']+df['ROC189']+df['ROC252'],3)

    #Finding the top 10%  rs_score stocks
    def rstop10(self):
        prices = self.getprices()
        rs_list = []
        for frame in prices:
            self.rs_line(frame)
            rs_list.append(frame.RS_line.tail(1).iloc[-1])
        rs_list.sort(reverse = True)
        i = int(round(len(rs_list)*0.1,0))
        top_10 = rs_list[i]
        return top_10    
            
        
    def VCP(self,df):
        df['SMA50'] = ta.trend.sma_indicator(df.Close, window=50)
        df['SMA150'] = ta.trend.sma_indicator(df.Close, window=150)
        df['SMA200'] = ta.trend.sma_indicator(df.Close, window=200)
        # calaulate the slope based on criteria 3, given 1month = 20 trading days
        df['SMA_slope_200'] = df['SMA200'].rolling(window = 20).apply(self.slope_reg)
        df['52_week_low'] = df['Close'].rolling(window =5*52).min()
        df['52_week_high'] = df['Close'].rolling(window =5*52).max()
        df['true_range_10d'] = (max(df['Close'][-10:-1])-min(df['Close'][-10:-1]))
        df['Turnover']= df['Close']*df['Volume']

        #Criteria 1 : Current price greater than 150 and 200-day expontenial moving average
        df['Criteria1'] = np.where((df['Close']>df['SMA150']) & (df['Close']>df['SMA200']),True,False)

        #Criteria 2 : The 150-day SMA must greater than 200-day EMA
        df['Criteria2'] = df['SMA150'] > df['SMA200']

        #Criteria 3 : The 200-day SMA must be trending up for at least 1 month.
        df['Criteria3'] = df['SMA_slope_200']>0.0

        #Criteria 4 : The 50-day SMA must greater than 150 & 200-day SMA
        df['Criteria4'] = np.where((df['SMA50']>df['SMA150']) & (df['SMA150']>df['SMA200']),True,False)

        #Criteria 5 : Current price greater than 50-day SMA
        df['Criteria5'] = df['Close']>df['SMA50']

        #Criteria 6: Current price must be 40% above 52-weeks low
        df['Criteria6'] = (df['Close']-df['52_week_low'])/df['52_week_low']> 0.4

        #Criteria 7: Current price must be within 15% of the 52 week high
        df['Criteria7'] = ((df['Close']-df['52_week_high'])/df['52_week_high'] < 0.15) & \
        ((df['Close']-df['52_week_high'])/df['52_week_high'] > -0.15)

        #Criteria 8: True Range in the last 10 days is less than 8% of current price(consolidation)
        df['Criteria8'] = df['true_range_10d'] < (df['Close']*0.08)

        #Criteria 9: RS rating withing 10%
        df['Criteria9'] = df['RS_line']> df['top_10']

        #Criteria 10: Turnover must be greater than 1million
        df['Criteria10'] = df['Turnover'] >= 1000000

        #Criteria 11: Current price must be greater than $1
        df['Criteria11'] = df['Close'] >1.0

        df['Fufillment'] = df[['Criteria1','Criteria2','Criteria3','Criteria4',\
                               'Criteria5','Criteria6','Criteria7','Criteria8',\
                               'Criteria9','Criteria10','Criteria11']].all(axis=1)

          
    def applytechnicals(self):
        top10 = self.rstop10()
        prices = self.getprices()
        for frame in prices:
            self.rs_line(frame)
            frame['top_10'] = top10
            self.VCP(frame)
        return prices

    def marketbreath(self):
        prices = self.applytechnicals()
        Ascending_list,Desending_list,New_high,New_low,C_20,C_50,C50_C150_C200,Fufillment_count = [],[],[],[],[],[],[],[]
        
        for price in prices:
            price['Asc'] = np.where((price.Close > price.Close.shift(1)),True,False)
            price['Des/UnChg'] = np.where((price.Close <= price.Close.shift(1)),True,False)
            price['New_high'] = np.where((price.Close >= price.Close.rolling(250).max()),True,False)
            price['New_low'] = np.where((price.Close <= price.Close.rolling(250).min()),True,False)
            price['c>20']= np.where((price.Close > price.Close.rolling(20).mean()),True,False)
            price['c20>c50']=np.where((price.Close.rolling(20).mean() >price.Close.rolling(50).mean()),True,False)
            
            Ascending_list.append(price['Asc'].iloc[-1])
            Desending_list.append(price['Des/UnChg'].iloc[-1])
            New_high.append(price['New_high'].iloc[-1])
            New_low.append(price['New_low'].iloc[-1])
            C_20.append(price['c>20'].iloc[-1])
            C_50.append(price['Criteria5'].iloc[-1])
            C50_C150_C200.append(price['Criteria4'].iloc[-1]) 
            Fufillment_count.append(price['Fufillment'].iloc[-1])
        
        total = sum(Ascending_list) + sum(Desending_list)
        data = {'Date:':date.today(),
            '# of Stocks monitored:': total,
            'Advanced / Declined Stocks:': f"{sum(Ascending_list)} / {sum(Desending_list)}",
            'Adv/Dec Ratio': sum(Ascending_list) / sum(Desending_list),
            '# of 52 weeks high Stocks:': sum(New_high),
            '# of 52 weeks low Stocks:': sum(New_low),
            'Percentage of stocks >SMA 20)': f"{round((sum(C_20)/total)*100,1)}%",
            'Percentage of stocks >SMA 50)': f"{round((sum(C_50)/total)*100,1)}%",
            'Percentage of stocks with SMA 50 > SMA 150 > SMA 200':f"{round((sum(C50_C150_C200)/total)*100,1)}%",
            '# of Stocks fits all criteria':sum(Fufillment_count),
            'Percentage of stocks fits all criteria ':f"{round((sum(Fufillment_count)/total)*100,1)}%"
        }
        df = pd.DataFrame.from_dict(data,orient='index')\
        .rename(columns={0: 'Daily Market Performance'}) 
        return df        
            
           
    
    def recommender(self):
        prices = self.applytechnicals()
        output = []
        for frame in prices:
            if frame['Fufillment'].tail(1).iloc[-1] == True:
                frame.Date =  pd.to_datetime(frame.Date)
                frame = frame.set_index('Date')
                mpf.plot(frame.loc['2022':'2023',:],type='candle',title=frame.Symbol.iloc[-1], \
                         style ='charles',volume = True, mav=(20,50,100),\
                         savefig=f"C:/Users/user/ChartTemporay/{date.today()}-{frame.Symbol.iloc[-1]}.png")
                output.append(frame[['Symbol','Close','52_week_low','52_week_high','RS_line']].tail(1))
        output = pd.concat(output).sort_values('RS_line',ascending=False).set_index('Symbol')
        return output
    
    #def create_pics_path(self):
     #   prices = self.applytechnicals()
      #  for frame in prices:
       #     if frame['Fufillment'].tail(1).iloc[-1] == True:
                

In [4]:
VCPinstance = VCP_Screener()

In [5]:
VCPinstance.updateDB()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
%run ./Desktop/creds.ipynb

In [7]:
#Create a list included all the daily file
prices = VCPinstance.applytechnicals()
path_of_images=[]
for frame in prices:
    if frame['Fufillment'].tail(1).iloc[-1] == True:
        path_of_images.append(f"C:/Users/user/ChartTemporay/{date.today()}-{frame.Symbol.iloc[-1]}.png")


In [8]:
from pathlib import Path
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

#path = "C:\Users\user\ChartTemporay"
#list_of_images = os.listdir(path)
sender = mymail
password = mypw
today = date.today()
marketbreath =VCPinstance.marketbreath()
recommender = VCPinstance.recommender()


msg  = MIMEMultipart()

msg['Subject'] = f"Daily Recommendatioans {today}"
  
html = """\
<html>
  <head>:</head>
  <body>
    <br>HK Stock Daily Market Breath<br>
    {0}
    <br>Stock Recommendation List<br>
    {1}
    
  </body>
</html>
""".format(marketbreath.to_html(),recommender.to_html())

msg.attach(MIMEText(html,'html'))


for filename in path_of_images:
    fp = open(filename, 'rb')
    msg_img = MIMEImage(fp.read())
    fp.close()
    msg_img.add_header('Content-ID', '<{}>'.format(filename))
    msg_img.add_header('Content-Disposition', 'inline', filename=os.path.basename(filename))
    msg.attach(msg_img)

server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()
server.login(sender,password)





(235, b'2.7.0 Accepted')

In [9]:
receiver = ['charleskfyuen@gmail.com','kinpong1000@hotmail.com',sender]

In [10]:
server.sendmail(sender,receiver,msg.as_string())

{}